In [1]:
!pip install -q keras
!pip install gym
!pip install “gym[atari]"

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
class Agent:

    def __init__(self):
        self.memory = []
        self.epsilon = 1.0 #exploration rate
        self.model = self.__model()

    def __model(self):
        model = Sequential()
        model.add(Dense(units=200,input_dim=40*40, activation='relu'))
        model.add(Dense(units=2, activation='sigmoid'))
        #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.compile(loss='mse', optimizer=Adam(lr=0.01, decay=0.01))

        return model

    def remember(self, state, action, reward, next_state, done):
        #store in memory the different states, actions, rewards...
        self.memory.append( (state, action, reward, next_state, done) )

    def replay(self, win):
        #fit model from memory
        gamma = 0.5 #importance of the next reward
        max_batch_size = 512

        #take care the memory could be big, so using minibatch
        minibatch = random.sample(self.memory, min(max_batch_size, len(self.memory)))
        list_x_batch, list_y_batch = [], []
        num_steps = len(minibatch)

        for state, action, reward, next_state, done in minibatch:

            '''re-check dimesions of array to avoid this evaluation'''
            if state.ndim != 2:
              continue
              
            target = self.model.predict(state)[0]

            if not done: #calculate discounted reward
                action_values = self.model.predict(next_state)[0]
                #following the formula of action-value expectation
                reward = reward + gamma * np.amax(action_values)
            #else: #PROVISIONAL
            #    map(lambda x:x+1, list_y_batch) #!! updating previous actions

            #customize the obtained reward with the calculated
            reg = (num_steps/(num_steps+1))
            end_reward =  reg if win else (reg-1)
            target[action] = reward + end_reward

            #append
            list_x_batch.append(state)
            list_y_batch.append(target)

        #train the model
        x_batch = np.vstack(list_x_batch)
        y_batch = np.vstack(list_y_batch)
        self.model.fit(x_batch, y_batch, verbose=0)

        #decrease exploration rate
        if self.epsilon > 0.01:
            self.epsilon *= 0.997


    def act(self, state):
        if self.epsilon > np.random.rand():
            return random.randint(0,1)

        #predict the action to do
        action_values = self.model.predict(state)[0]

        return np.argmax(action_values)

In [10]:
import os
import gym
import random
import numpy as np
from time import sleep
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam


def prepro(I):
# prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector
  I = I[35:195] # crop
  I = I[::4,::4,0] # downsample by factor of 4
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()


# code for the two only actions in Pong
UP_ACTION = 2
DOWN_ACTION = 3

# initializing our environment
env = gym.make("Pong-v0")

# beginning of an episode
observation = env.reset()
observation = prepro(observation).reshape(1, -1)

# model weights
h5file = "weights.h5"

# agent
agent = Agent()

# get model
if os.path.exists(h5file):
    agent.model.load_weights(h5file)


# training conf
training = True
#x_train, y_train, rewards = [], [], []
#reward_sum = 0

# main loop
for i in range(10000000):
    # predict action
    '''re-check dimesions of array to avoid this evaluation'''
    if observation.ndim != 2:
      movement = UP_ACTION
    else:
      action = agent.act(observation)
      movement = UP_ACTION if action==0 else DOWN_ACTION

    # do one step
    next_observation, reward, done, info = env.step(movement)

    #row vector
    ##print("reward:{} done:{} info:{}".format(reward, done, info))
    next_observation = prepro(next_observation).reshape(1, -1)
    #next_observation = next_observation.reshape(1, -1)

    #save the current observation
    agent.remember(observation, action, reward, next_observation, done)

    #update state
    observation = next_observation

    if done:
        if training:
            win = True if reward == 1 else False
            agent.replay(win)
            agent.model.save_weights(h5file)
            
        if reward==1:
          print("Win!!")
          break
        else:
          print("Lose..")
          
        print(info)
        observation = env.reset()


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.lives': 0}
Lose..
{'ale.liv

KeyboardInterrupt: ignored

In [0]:
!ls weights.h5

weights.h5
